In [46]:
import numpy as np 
import pandas as pd

In [48]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import pandas as pd
pd.set_option("display.max_columns", None)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error

In [70]:
# Load dataset
df = pd.read_csv("stg_sales.csv")

In [72]:
# Convert date column to datetime
df["date"] = pd.to_datetime(df["date"])

In [74]:
df["Average Price"] = (df["amount"] / df["boxes_shipped"]).round()

In [76]:
df_product = df.pivot_table(index = "date", columns = "product", values = "Average Price").stack().reset_index()
df_product = df_product.rename(columns = {0 : "Average Sales Price"})
df_product.head()

,date,product,Average Sales Price
0,2022-01-03,99% Dark & Pure,33.0
1,2022-01-03,Caramel Stuffed Bars,21.0
2,2022-01-03,Raspberry Choco,75.0
3,2022-01-03,Smooth Sliky Salty,22.0
4,2022-01-03,Spicy Special Slims,10.0


In [78]:
df_new = pd.merge(df, df_product, how = "left", on = ["date", "product"])
df_new.head()


,sales_person,country,product,date,amount,boxes_shipped,Average Price,Average Sales Price
0,Jehu Rudeforth,UK,Mint Chip Choco,2022-01-04,5320,180,30.0,30.0
1,Van Tuxwell,India,85% Dark Bars,2022-08-01,7896,94,84.0,102.5
2,Gigi Bohling,India,Peanut Butter Cubes,2022-07-07,4501,91,49.0,49.0
3,Jan Morforth,Australia,Peanut Butter Cubes,2022-04-27,12726,342,37.0,37.0
4,Jehu Rudeforth,UK,Peanut Butter Cubes,2022-02-24,13685,184,74.0,74.0


In [80]:
df_new["Diff"] = df_new["Average Sales Price"] - df_new["Average Price"]
df_new.head()

,sales_person,country,product,date,amount,boxes_shipped,Average Price,Average Sales Price,Diff
0,Jehu Rudeforth,UK,Mint Chip Choco,2022-01-04,5320,180,30.0,30.0,0.0
1,Van Tuxwell,India,85% Dark Bars,2022-08-01,7896,94,84.0,102.5,18.5
2,Gigi Bohling,India,Peanut Butter Cubes,2022-07-07,4501,91,49.0,49.0,0.0
3,Jan Morforth,Australia,Peanut Butter Cubes,2022-04-27,12726,342,37.0,37.0,0.0
4,Jehu Rudeforth,UK,Peanut Butter Cubes,2022-02-24,13685,184,74.0,74.0,0.0


In [82]:
Q1 = df_new["Diff"].quantile(0.25)
Q3 = df_new["Diff"].quantile(0.75)

IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_renew = df_new[(df_new["Diff"] >= lower_bound) & (df_new["Diff"] <= upper_bound)]
df_renew

,sales_person,country,product,date,amount,boxes_shipped,Average Price,Average Sales Price,Diff
0,Jehu Rudeforth,UK,Mint Chip Choco,2022-01-04,5320,180,30.0,30.0,0.0
2,Gigi Bohling,India,Peanut Butter Cubes,2022-07-07,4501,91,49.0,49.0,0.0
3,Jan Morforth,Australia,Peanut Butter Cubes,2022-04-27,12726,342,37.0,37.0,0.0
4,Jehu Rudeforth,UK,Peanut Butter Cubes,2022-02-24,13685,184,74.0,74.0,0.0
5,Van Tuxwell,India,Smooth Sliky Salty,2022-06-06,5376,38,141.0,141.0,0.0
...,...,...,...,...,...,...,...,...,...
1089,Karlen McCaffrey,Australia,Spicy Special Slims,2022-05-17,4410,323,14.0,14.0,0.0
1090,Jehu Rudeforth,USA,White Choc,2022-06-07,6559,119,55.0,55.0,0.0
1091,Ches Bonnell,Canada,Organic Choco Syrup,2022-07-26,574,217,3.0,3.0,0.0
1092,Dotty Strutley,India,Eclairs,2022-07-28,2086,384,5.0,5.0,0.0


In [84]:
df_renew["date"] = df_renew["date"].astype("str")
df_renew[["Year", "Month", "Day"]] = df_renew["date"].str.split("-", expand = True)
df_renew["Year"] = df_renew["Year"].astype("int")
df_renew["Month"] = df_renew["Month"].astype("int")
df_renew["Day"] = df_renew["Day"].astype("int")
df_renew.drop("date", axis = 1, inplace = True)

In [86]:
df_renew.drop(["Average Price", "Diff"], axis = 1, inplace = True)
df_renew.head()

,sales_person,country,product,amount,boxes_shipped,Average Sales Price,Year,Month,Day
0,Jehu Rudeforth,UK,Mint Chip Choco,5320,180,30.0,2022,1,4
2,Gigi Bohling,India,Peanut Butter Cubes,4501,91,49.0,2022,7,7
3,Jan Morforth,Australia,Peanut Butter Cubes,12726,342,37.0,2022,4,27
4,Jehu Rudeforth,UK,Peanut Butter Cubes,13685,184,74.0,2022,2,24
5,Van Tuxwell,India,Smooth Sliky Salty,5376,38,141.0,2022,6,6


In [88]:
ohe = OneHotEncoder(sparse_output=False)
encoded_data = ohe.fit_transform(df_renew[cat_cols])


df_encoded = pd.DataFrame(encoded_data, columns = ohe.get_feature_names_out(cat_cols))
df_renew = df_renew.drop(columns = cat_cols)

df_final = pd.concat([df_renew, df_encoded], axis=1)
df_final

,amount,boxes_shipped,Average Sales Price,Year,Month,Day,sales_person_Andria Kimpton,sales_person_Barr Faughny,sales_person_Beverie Moffet,sales_person_Brien Boise,sales_person_Camilla Castle,sales_person_Ches Bonnell,sales_person_Curtice Advani,sales_person_Dennison Crosswaite,sales_person_Dotty Strutley,sales_person_Gigi Bohling,sales_person_Gunar Cockshoot,sales_person_Husein Augar,sales_person_Jan Morforth,sales_person_Jehu Rudeforth,sales_person_Kaine Padly,sales_person_Karlen McCaffrey,sales_person_Kelci Walkden,sales_person_Madelene Upcott,sales_person_Mallorie Waber,sales_person_Marney O'Breen,sales_person_Oby Sorrel,sales_person_Rafaelita Blaksland,sales_person_Roddy Speechley,sales_person_Van Tuxwell,sales_person_Wilone O'Kielt,country_Australia,country_Canada,country_India,country_New Zealand,country_UK,country_USA,product_50% Dark Bites,product_70% Dark Bites,product_85% Dark Bars,product_99% Dark & Pure,product_After Nines,product_Almond Choco,product_Baker's Choco Chips,product_Caramel Stuffed Bars,product_Choco Coated Almonds,product_Drinking Coco,product_Eclairs,product_Fruit & Nut Bars,product_Manuka Honey Choco,product_Milk Bars,product_Mint Chip Choco,product_Orange Choco,product_Organic Choco Syrup,product_Peanut Butter Cubes,product_Raspberry Choco,product_Smooth Sliky Salty,product_Spicy Special Slims,product_White Choc
0,5320.0,180.0,30.0,2022.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4501.0,91.0,49.0,2022.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,12726.0,342.0,37.0,2022.0,4.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,13685.0,184.0,74.0,2022.0,2.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,5376.0,38.0,141.0,2022.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
778,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
779,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
780,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [90]:
df_final_2 = df_final.copy()
df_final.drop("amount", axis = 1, inplace = True)
df_final = pd.concat([df_final, df_final_2["amount"]], axis = 1)
df_final.head()

,boxes_shipped,Average Sales Price,Year,Month,Day,sales_person_Andria Kimpton,sales_person_Barr Faughny,sales_person_Beverie Moffet,sales_person_Brien Boise,sales_person_Camilla Castle,sales_person_Ches Bonnell,sales_person_Curtice Advani,sales_person_Dennison Crosswaite,sales_person_Dotty Strutley,sales_person_Gigi Bohling,sales_person_Gunar Cockshoot,sales_person_Husein Augar,sales_person_Jan Morforth,sales_person_Jehu Rudeforth,sales_person_Kaine Padly,sales_person_Karlen McCaffrey,sales_person_Kelci Walkden,sales_person_Madelene Upcott,sales_person_Mallorie Waber,sales_person_Marney O'Breen,sales_person_Oby Sorrel,sales_person_Rafaelita Blaksland,sales_person_Roddy Speechley,sales_person_Van Tuxwell,sales_person_Wilone O'Kielt,country_Australia,country_Canada,country_India,country_New Zealand,country_UK,country_USA,product_50% Dark Bites,product_70% Dark Bites,product_85% Dark Bars,product_99% Dark & Pure,product_After Nines,product_Almond Choco,product_Baker's Choco Chips,product_Caramel Stuffed Bars,product_Choco Coated Almonds,product_Drinking Coco,product_Eclairs,product_Fruit & Nut Bars,product_Manuka Honey Choco,product_Milk Bars,product_Mint Chip Choco,product_Orange Choco,product_Organic Choco Syrup,product_Peanut Butter Cubes,product_Raspberry Choco,product_Smooth Sliky Salty,product_Spicy Special Slims,product_White Choc,amount
0,180.0,30.0,2022.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5320.0
2,91.0,49.0,2022.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4501.0
3,342.0,37.0,2022.0,4.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,12726.0
4,184.0,74.0,2022.0,2.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13685.0
5,38.0,141.0,2022.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5376.0


In [92]:
df_final.dropna(axis = 0, inplace = True)
df_final

,boxes_shipped,Average Sales Price,Year,Month,Day,sales_person_Andria Kimpton,sales_person_Barr Faughny,sales_person_Beverie Moffet,sales_person_Brien Boise,sales_person_Camilla Castle,sales_person_Ches Bonnell,sales_person_Curtice Advani,sales_person_Dennison Crosswaite,sales_person_Dotty Strutley,sales_person_Gigi Bohling,sales_person_Gunar Cockshoot,sales_person_Husein Augar,sales_person_Jan Morforth,sales_person_Jehu Rudeforth,sales_person_Kaine Padly,sales_person_Karlen McCaffrey,sales_person_Kelci Walkden,sales_person_Madelene Upcott,sales_person_Mallorie Waber,sales_person_Marney O'Breen,sales_person_Oby Sorrel,sales_person_Rafaelita Blaksland,sales_person_Roddy Speechley,sales_person_Van Tuxwell,sales_person_Wilone O'Kielt,country_Australia,country_Canada,country_India,country_New Zealand,country_UK,country_USA,product_50% Dark Bites,product_70% Dark Bites,product_85% Dark Bars,product_99% Dark & Pure,product_After Nines,product_Almond Choco,product_Baker's Choco Chips,product_Caramel Stuffed Bars,product_Choco Coated Almonds,product_Drinking Coco,product_Eclairs,product_Fruit & Nut Bars,product_Manuka Honey Choco,product_Milk Bars,product_Mint Chip Choco,product_Orange Choco,product_Organic Choco Syrup,product_Peanut Butter Cubes,product_Raspberry Choco,product_Smooth Sliky Salty,product_Spicy Special Slims,product_White Choc,amount
0,180.0,30.0,2022.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5320.0
2,91.0,49.0,2022.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4501.0
3,342.0,37.0,2022.0,4.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,12726.0
4,184.0,74.0,2022.0,2.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13685.0
5,38.0,141.0,2022.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5376.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,8.0,238.0,2022.0,8.0,22.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1904.0
793,275.0,18.0,2022.0,4.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4844.0
794,313.0,31.0,2022.0,5.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9625.0
795,323.0,14.0,2022.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4599.0


In [94]:
X = df_final.iloc[:, :-1]
y = df_final.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [96]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [98]:
xgbr = XGBRegressor()
param_grid = {"n_estimators" : [20,40,60,80,100], "max_depth" : [2,3,4,5], "min_samples_split" : [2,3,4], "max_features" : [0.2,0.4,0.6,0.8,1.0]}

gscv = GridSearchCV(xgbr, param_grid = param_grid, cv = 5, )
gscv.fit(X_train, y_train)

print("Best parameters :", gscv.best_params_)

Best parameters : {'max_depth': 4, 'max_features': 0.2, 'min_samples_split': 2, 'n_estimators': 100}


In [100]:
xgbr = XGBRegressor(n_estimators = 100, max_depth = 4, min_samples_split = 2, max_features = 0.2)
xgbr.fit(X_train, y_train)

y_xgbr = xgbr.predict(X_test).round()
print("MAPE Score :", mean_absolute_percentage_error(y_test, y_xgbr))

MAPE Score : 0.1823067765548857


In [ ]:
# Save the trained model
joblib.dump(model, "xgboost_model.pkl")